In [ ]:
import json
import skimage

In [ ]:
data_dir = 'data/'
template_dir = 'templates/'
moon_dir = data_dir+'moon/'
moon_template_dir = template_dir+'moon/'

In [ ]:
with open(moon_dir+'moon_metadata.json') as metadata:
    moon_metadata = json.load(metadata)